## 仅供交叉验证 规则模型-按总量分类

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from utils import *
from sklearn.model_selection import StratifiedKFold

np.random.seed(42)
random.seed(42)
n_input = 11

读取数据

In [6]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [7]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


## 规范数据并获取5折交叉检验所需的训练集和验证集

In [8]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 按总量切分数据（非均等切分）

In [2]:
def split_data_by_es(data, targets):
    total_es = np.sum(data[:, :11, -2], axis=1)
    q75, q25 = np.percentile(total_es, [75 ,25])
    iqr = q75 - q25
    upper_bond = q75 + 1.5 * iqr
    lower_bond = q25 - 1.5 * iqr
    
    data1, target1 = data[np.argwhere(total_es > upper_bond).reshape(-1)], targets[np.argwhere(total_es > upper_bond).reshape(-1)]
    data2, target2 = data[np.argwhere((q75 < total_es) & (total_es <= upper_bond)).reshape(-1)], targets[np.argwhere((q75 < total_es) & (total_es <= upper_bond)).reshape(-1)]
    data3, target3 = data[np.argwhere((q25 < total_es) & (total_es <= q75)).reshape(-1)], targets[np.argwhere((q25 < total_es) & (total_es <= q75)).reshape(-1)]
    data4, target4 = data[np.argwhere(total_es <= q25).reshape(-1)], targets[np.argwhere(total_es <= q25).reshape(-1)]
    
    return data1, target1, data2, target2, data3, target3, data4, target4

### 构建模型，无需训练

In [3]:
def enhanced_persistence_model_predict(X, y):
    es_x = X[:, -1, -2].reshape(len(X), 1)
    es_y = y[:, :-1]
    return np.hstack((es_x, es_y))

def persistence_model_predict(X, y):
    timesteps = y.shape[1]
    result = np.zeros((X.shape[0], timesteps))
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            result[i][j] = X[i, -1, -2]
    return result

In [4]:
def cross_validation(X, y, y_cat, kfold, scaler):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[]
    }

    for train, test in kfold.split(X, y_cat):
        X_test = X[test]
        y_test = y[test]
        
        # 按总量划分数据集
        X_test1, y_test1, X_test2, y_test2, X_test3, y_test3, X_test4, y_test4 = split_data_by_es(X_test, y_test)
        test_xs = [X_test1, X_test2, X_test3, X_test4]
        test_ys = [y_test1, y_test2, y_test3, y_test4]
        i_s = [1, 2, 3, 4]
        
        # 预测，规则模型无需训练
        y_test = []
        y_pred = []
        for i in range(len(i_s)):
            y_test.append(test_ys[i])
            y_pred.append(persistence_model_predict(test_xs[i], test_ys[i]).reshape(test_ys[i].shape))
        
        y_test = np.concatenate(y_test)
        y_pred = np.concatenate(y_pred)
        
#         y_pred = persistence_model_predict(X_test, y_test).reshape(y_test.shape)

        metrics = ['mae', 'rmse','ndcg']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics

In [5]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler)
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
    pickle.dump(metrics, open('persistence_metrics.dict', 'wb'))
    
    return metrics

In [6]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


In [8]:
metrics

{'gene': {'overall': {'mae': 0.9953142293401906,
   'rmse': 1.6622467374475698,
   'ndcg': 0.43682902822305286},
  'annual': {'mae': array([0.41172305, 0.61018843, 0.96843844, 1.32754986, 1.65867136]),
   'rmse': array([0.9055439 , 1.13584225, 1.5909463 , 1.95090073, 2.30408695]),
   'ndcg': array([0.51037426, 0.4024734 , 0.17369096, 0.20031431, 0.1710085 ])}},
 'transplant': {'overall': {'mae': 0.9809174394788709,
   'rmse': 1.6180250936731273,
   'ndcg': 0.08145837273935752},
  'annual': {'mae': array([1.01396299, 0.96167129, 0.94218809, 0.9609344 , 1.02583042]),
   'rmse': array([1.7680078 , 1.60303796, 1.55425346, 1.52053965, 1.6243132 ]),
   'ndcg': array([0.00654271, 0.08906358, 0.02624443, 0.13302272, 0.06534455])}}}